### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 3 sobre los servicios web de la EMT creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga las siguientes páginas:

*  Una página principal que mostrará un conjunto de varios enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * Servicio 1:Servicio de obtener la ruta optima entre dos destinos usando los servicios de la EMT
   
   * Servicio 2:Servicio de mostrar los lugares de interés entre dos destinos indicados.
   
   * Servicio 3:Servicio de mostrar las paradas de autobús más cercanas con toda la información acerca de la parada: líneas que pasan, frecuencia, ...
   
* Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la descripción de la ruta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

* Cuando el usuario pulsa sobre el servicio 2 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará un listado con los lugares de interés recuperados en la ruta entre las dos calles. De cada resultado mostrará la información disponible: nombre, teléfono, calle,...En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial.[3 puntos]

* Este servicio es nuevo, y requiere el uso de otro servicio web de la EMT. En este caso, se va a utilizar el servicio GetStreet de GEO. Cuando el usuario pulsa sobre el servicio 3 se le mostrará un formulario en el que podrá introducir el nombre de una calle y un número de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la información de las paradas más cercanas a la calle con toda la información acerca de las líneas de autobuses que paran en esas paradas. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

## Normas de entrega

* Fecha tope de entrega: 22/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [1]:
#Código práctica 3

import requests
from xml.etree import ElementTree

def get_street (name, number):
    """Devuelve la informacion de una calle dado su nombre y su numero.
    
    Parametros de entrada:
    name --- nombre de la calle
    number --- numero de la calle
    
    Retorno: información asociada a la calle
    """
    
    data = {
        'idClient':'WEB.SERV.gabriese@ucm.es ',
        'PassKey':'68020487-D67D-4A72-A362-32F6AAB4BB33',
        'description': name,
        'streetNumber':number,
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }
    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    return requests.post(url, data=data).text
    
def save_XML(name, content):
    """Crea un fichero y vuelca en el una cadena con un xml
    
    Parametros de entrada:
    name --- nombre del fichero a crear
    content --- contenido a volcar en el fichero    
    """
    pointer = open(name +'.xml','w')
    pointer.write(content)
    pointer.close()

def parse_XML(name):
    """Devuelve la representación en forma de arbol de un fichero xml
    
    Parametros de entrada:
    name --- nombre del fichero
    
    Retorno:
    result --- representacion del xml
    """
    
    f = open(name + '.xml', "rt")
    result = ElementTree.parse(f)
    f.close()
    return result

def get_coordinates(name,number):
    """Devuelve las coordenadas X e Y de una calle.
    
    Parametros de entrada:
    name --- nombre de la calle
    number ---  numero de la calle    
    """
    
    response = get_street(name,number)
    save_XML('result',response)
    tree = parse_XML("result")
    return tree.find('Site').find('CoordinateX').text, tree.find('Site').find('CoordinateY').text


def ask_route_service(origin_X, origin_Y, destiny_X, destiny_Y):
    """Devuelve la informacion de la ruta optima entre dos localizaciones
    
    Parametros de entrada:
    origin_X --- coordenada x del origen
    origin_Y --- coordenada y del origen
    destiny_X --- coordenada x del destino
    destiny_Y --- coordenada y del destino
    
    Retorno: informacion asociada a la ruta
    """
    
    data = {
        'idClient':'WEB.SERV.gabriese@ucm.es',
        'PassKey':'68020487-D67D-4A72-A362-32F6AAB4BB33',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': origin_X,
        'coordinateYFrom':origin_Y,
        'originName':'',
        'coordinateXTo':destiny_X,
        'coordinateYTo':destiny_Y,
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }
    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    return requests.post(url, data=data).text
    
def get_route_result(xml_file):
    """Imprime por pantalla los datos del fichero xml proporcionado
    
    Parametros de entrada:
    xml_file --- fichero con los datos a mostrar
    """
    
    save_XML('result', xml_file)
    tree = parse_XML("result")
    
    result= list()
    
    
    
    result.append(tree.find("ListRouteData").find("RouteData").find("DescriptionRouteData").find("DescriptionDate").text)
    result.append(tree.find("ListRouteData").find("RouteData").find("DescriptionRouteData").find("DescriptionInitTime").text)
    result.append("Hora estimada de llegada : " + tree.find("ListRouteData").find("RouteData").find("DescriptionRouteData").find("DescriptionEstimateTimeArrival").text)
    result.append("Transbordos:" + tree.find("ListRouteData").find("RouteData").find("DescriptionRouteData").find("Transfers").text)
    result.append("Duración:" + tree.find("ListRouteData").find("RouteData").find("DescriptionRouteData").find("LongJourney").text)
    result.append("Descripcion textual de la ruta:")
    
    for node in tree.iter("Section"):
        if not (node.find("WalkingLeg") is None):
            result.append(node.find("WalkingLeg").find("SourceWalkingLeg").find("RouteDescription").text)
        if not (node.find("BusLeg") is None):
            result.append(node.find("BusLeg").find("SourceBusLeg").find("RouteDescription").text)  
    return result

# FUNCION SOLICITADA EN EL ENUNCIADO
def ask_show_route(origin_X, origin_Y, destiny_X, destiny_Y):
    """Imprime por pantalla la informacion de una ruta dadas las coordenadas de origen-destino
    
    Parametros de entrada:
    origin_X --- coordenada x del origen
    origin_Y --- coordenada y del origen
    destiny_X --- coordenada x del destino
    destiny_Y --- coordenada y del destino
    """
    
    xml_file = ask_route_service(origin_X, origin_Y, destiny_X, destiny_Y)
    return get_route_result(xml_file)




In [2]:
#Pruebas




In [3]:
       


def get_POI_data(xml_file):
    """Devuelve los datos asociados a los puntos de interes de una ruta
    
    Parametros de entrada:
    xml_file --- fichero xml con los puntos de interes
    
    Retorno:
    dic --- diccionario con el nombre, localizacion y telefono de los puntos de interes
    """
    
    # procesamos el xml para poder trabajarlo
    save_XML('result', xml_file)
    tree = parse_XML("result")
    
    dic = {}
    print("\n\nListado de puntos de interes: ")
    for node in tree.iter("POI"):
        # mensajes por defecto si los campos estan vacios
        direccion = "no hay datos para este campo"
        telefono = "no hay datos para este campo"
        """Las siguientes sentencias if son necesarias para procesar etiquetas vacias y etiquetas
        que contienen espacios y saltos de lineas. En el caso de las vacias basta con la comprobacion 
        == "None" mientras que en el caso de los saltos de linea y espacios hacemos strip() sobre ese
        texto y comparamos con la cadena vacia.
        """
        if not (str(node.find("direccion").text).strip() == "None" or 
                str(node.find("direccion").text).strip() == ""):
            direccion = str(node.find("direccion").text).strip() 
        if not (str(node.find("telefono").text).strip()== "None" or 
                str(node.find("telefono").text).strip() == ""):
            telefono = str(node.find("telefono").text).strip() 
        if not (str(node.find("nombre").text).strip() == "None" or 
                str(node.find("nombre").text).strip() == ""):
            print(node.find("nombre").text)
            dic.setdefault(node.find("Id").text, [node.find("nombre").text.strip(), direccion, telefono])
    return dic

def show_POI_data(dic):
    """Pide al usuario un punto de interes y muestra sus datos si pertenece a la ruta
    
    Parametros de entrada:
    dic --- diccionario con la informacion de los puntos de interes
    """

    
    print("\n\nIntroduce el nombre de un punto de interes: ")
    poi = str(input())
    if poi.isdigit(): # si introducen un numero levantamos excepcion, el resto de casos son del if-else
        raise ValueError
    aux_data = None
    for (key, data) in dic.items():
        if data[0] == poi:
            aux_data = data
            break
    if not (aux_data is None):
        print("\n\nDatos del punto de interes elejido: ")
        print("\t- Nombre del punto de interés:", aux_data[0])
        print("\t- Dirección del punto de interés:", aux_data[1])
        print("\t- Teléfono del punto de interés:", aux_data[2])
    else:
        print("\n\nEl punto de interes proporcionado no corresponde al recorrido.")
        
# FUNCION SOLICITADA EN EL ENUNCIADO
def get_POI_from_route():
    """Pide dos localizaciones y muestra la lista de POI en la ruta entre dichas localizaciones
    """
    try:
        # pedimos las calles al usuario y obtenemos las coordenadas
        O_street, O_number, D_street, D_number = ask_street()
        origin_X, origin_Y = get_coordinates(O_street, O_number)
        destiny_X, destiny_Y = get_coordinates(D_street, D_number)
        # obtenemos el xml, generamos la lista de POI y mostramos su informacion
        xml_file = ask_route_service(origin_X, origin_Y, destiny_X, destiny_Y)
        dic = get_POI_data(xml_file)
        show_POI_data(dic)
    except Exception:
        print("Los datos proporcionados no son validos")

In [4]:
#MEJORAR LA INTERFAZ
#TRATAR EXCEPCIONES


#Código práctica 6
from bottle import route, run, get, post, template, request

@get ('/')
def initial_site():
    return template('initial.tpl')

@get ('/apartado1')
def show_form():
    return template('apartado1.tpl')

@post('/apartado1')
def do_login():
    o_street= request.forms.get('o_street')
    o_number= request.forms.get('o_number')
    d_street= request.forms.get('d_street')
    d_number= request.forms.get('d_number')
    ox,oy= get_coordinates(o_street,o_number)
    dx,dy= get_coordinates(d_street,d_number)
    result=ask_show_route(ox,oy,dx ,dy)
    
    return template('apartado1_result.tpl',texto=result)

@get('/apartado2')
    return template('apartado2.tpl')
@post('/apartado2')
    o_street= request.forms.get('o_street')
    o_number= request.forms.get('o_number')
    d_street= request.forms.get('d_street')
    d_number= request.forms.get('d_number')
    ox,oy= get_coordinates(o_street,o_number)
    dx,dy= get_coordinates(d_street,d_number)
    result=ask_show_route(ox,oy,dx ,dy)
    xml_file = ask_route_service(ox, oy, dx, dy)
    dic = get_POI_data(xml_file)
    result=get_POI_data(dic)

run(host='localhost', port=8080)



Bottle v0.13-dev server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [17/Nov/2017 13:43:08] "GET / HTTP/1.1" 200 388
127.0.0.1 - - [17/Nov/2017 13:43:09] "GET /apartado1 HTTP/1.1" 200 559
127.0.0.1 - - [17/Nov/2017 13:43:38] "POST /apartado1 HTTP/1.1" 200 936
127.0.0.1 - - [17/Nov/2017 13:44:40] "GET / HTTP/1.1" 200 388
127.0.0.1 - - [17/Nov/2017 13:44:42] "GET /apartado1 HTTP/1.1" 200 559
